In [2]:
import pdfplumber
import pprint
import re

# service_code_re = re.compile = (r'\[([^\]]*)\]')

with pdfplumber.open('pdf/WO_283650.pdf') as pdf:
    p0 = pdf.pages[0]
    
    lines = p0.extract_text()
    print(lines)
    for line_id, line in enumerate(lines.split('\n')):
        # get Work Order ID
        if line.startswith('Work Order ID'):
            work_order_id_plus = line.split('Work Order ID: ')[1]
            work_order_id = work_order_id_plus.split(' Work Order Date: ')[0]
#             print(work_order_id)
        
        # get Service Code
        if line.startswith('Service Code'):
            print(line)
#             service_code = service_code_re.search(line)
#             print(service_code)
        
        
        
        print(line_id, line)

#     tables = p0.extract_tables()
#     for table in tables:
#         pprint.pprint(table)
    
#     im = p0.to_image()
#     im.debug_tablefinder()
#     im.save('/Users/jpheasly/Development/homeless-cleanups/foo.png', format="PNG")
    

Work Order ID: 283650 Work Order Date: 08/30/19 11:04:16 AM
Service Code: [SWM30] SWIM Camp/Debris Cleanup Assigned To: Stehlar, Doug
Service Address:
SEE PIN LOCATION
Latitude: Longitude:
44.051902247761 -123.124606348008
Caller Information:
Work Description:
SEE PIN LOCATION
Action Notes:
Action Date Status Action By Action Notes
08/30/19 11:04:16 AM Assigned Schneider, Erick Work Order Created
Camp abandoned, trash
cleaned up x2 bags. Work
09/03/19 10:06:27 AM Closed Stehlar, Doug
Order completed by Doug
Stehlar.
Automatically reassigned to
04/02/20  7:39:23 AM Complete Hallett, Jackie
Stehlar, Doug by the system.
Result Summary:
Activity Type Resource Qty Units Performed On
Post Camp - 24  Employee 
Schneider, Erick
Hour Notice Hours 0.25 Hours 08/30/2019
Evens, Jacob
0.25 Hours 08/30/2019
Equipment 
(504035) 2006 GMC W3500
Hours 0.25 Hours 08/30/2019
Activity Notes:
Activity Performed On Notes
Posted camp in ROW. Green and 
Post Camp - 24 Hour Notice 08/30/2019
tan tent.
Image Att